Unzip

In [ ]:
!unzip Localization_project-main.zip
%ls

Archive:  Localization_project-main.zip
   creating: Localization_project-main/
   creating: Localization_project-main/Ground/
   creating: Localization_project-main/Ground/all_gt/
  inflating: Localization_project-main/Ground/all_gt/1.csv  
  inflating: Localization_project-main/Ground/all_gt/10.csv  
  inflating: Localization_project-main/Ground/all_gt/11.csv  
  inflating: Localization_project-main/Ground/all_gt/12.csv  
  inflating: Localization_project-main/Ground/all_gt/13.csv  
  inflating: Localization_project-main/Ground/all_gt/14.csv  
  inflating: Localization_project-main/Ground/all_gt/15.csv  
  inflating: Localization_project-main/Ground/all_gt/16.csv  
  inflating: Localization_project-main/Ground/all_gt/17.csv  
  inflating: Localization_project-main/Ground/all_gt/18.csv  
  inflating: Localization_project-main/Ground/all_gt/19.csv  
  inflating: Localization_project-main/Ground/all_gt/2.csv  
  inflating: Localization_project-main/Ground/all_gt/20.csv  
  inflating: Lo

RSSI_Brute force

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, Kernel, Hyperparameter, RBF, ConstantKernel, RationalQuadratic, Matern, ExpSineSquared, DotProduct
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean, cdist
import scipy

# Utility to split data into train and test sets
def testAndTrainData(df1, df2):
    tmp1 = df1['location'].to_numpy()
    y1 = df1['rssi_mean']
    y_train = y1.to_numpy()
    X1 = [list(map(int, i.split(","))) for i in tmp1]
    X_train = np.array(X1)

    tmp2 = df2['location'].to_numpy()
    y2 = df2['rssi_mean']
    y_test = y2.to_numpy()
    X2 = [list(map(int, i.split(","))) for i in tmp2]
    X_test = np.array(X2)

    return X_train, y_train, X_test, y_test

def hybridGPKernal():
    long_term_trend_kernel = 50.0 ** 2 * RBF(length_scale=50.0)
    seasonal_kernel = (
        2.0 ** 2
        * RBF(length_scale=100.0)
        * ExpSineSquared(length_scale=1.0, periodicity=1.0, periodicity_bounds="fixed")
    )

    irregularities_kernel = 0.5 ** 2 * RationalQuadratic(length_scale=1.0, alpha=1.0)

    noise_kernel = 0.1 ** 2 * RBF(length_scale=0.1) + WhiteKernel(
        noise_level=0.1 ** 2, noise_level_bounds=(1e-5, 1e5)
    )

    rssi_kernel = (
        long_term_trend_kernel + seasonal_kernel + irregularities_kernel + noise_kernel
    )

    return rssi_kernel

# Epanechnikov kernel for GPR
class EpanechnikovKernel(Kernel):
    def __init__(self, length_scale=1.0, length_scale_bounds=(1e-5, 1e5)):
        self.length_scale = length_scale
        self.length_scale_bounds = length_scale_bounds

    @property
    def hyperparameter_length_scale(self):
        return Hyperparameter("length_scale", "numeric", self.length_scale_bounds)

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        length_scale = self.length_scale

        if Y is None:
            Y = X
        else:
            Y = np.atleast_2d(Y)

        # Pairwise Euclidean distances
        dists_sq = np.sum(X**2, axis=1)[:, np.newaxis] + \
                   np.sum(Y**2, axis=1)[np.newaxis, :] - \
                   2 * np.dot(X, Y.T)
        dists = np.sqrt(np.maximum(dists_sq, 1e-12))
        dists_normalized = dists / length_scale

        # Epanechnikov kernel
        K = (3 / 4) * np.maximum(1 - dists_normalized**2, 0)

        if eval_gradient:
            mask = (dists_normalized <= 1)
            dK_dl = (3 / 4) * (2 * dists_normalized**2 / length_scale) * mask
            return K, dK_dl[..., np.newaxis]
        else:
            return K

    def diag(self, X):
        return np.full(X.shape[0], 0.75)

    def is_stationary(self):
        return True

    def __repr__(self):
        return f"EpanechnikovKernel(length_scale={self.length_scale})"

def calculate_distances(X):
    """Calculate distances from origin and center for enhanced features"""
    center = np.array([97.5, 115.5])  # Center of the grid (242x259)
    distances = np.array([
        [euclidean(point, [0, 0]), euclidean(point, center)]
        for point in X
    ])
    return distances



# Main processing loop
floor = '/content/Localization_project-main/Third'
folder_path = floor + '/beacons_gt/'

test_location = [
    "72, 55","55, 38","104, 55","72, 155","72, 174","44, 191","31, 32","171, 227","111, 25","171, 151","171, 140",
"118, 46","153, 55","194, 230","203, 214","79, 24","24, 38","55, 55","118, 41","72, 143","108, 191","194, 222","115, 191"
]

seasonal_kernel = (2.0 ** 2 * RBF(length_scale=100.0) * ExpSineSquared(length_scale=1.0, periodicity=1.0, periodicity_bounds="fixed"))
irregularities_kernel = 0.5 ** 2 * RationalQuadratic(length_scale=1.0, alpha=1.0)

# Kernel candidates to test
kernel_candidates = [
    # ConstantKernel() * Matern(nu=0.5) + WhiteKernel(),
    # ConstantKernel() * Matern(nu=1.5) + WhiteKernel(),
    # ConstantKernel() * Matern(nu=2.5) + WhiteKernel(),
    # ConstantKernel() * RBF() + WhiteKernel(),
    # ConstantKernel() * RationalQuadratic() + WhiteKernel(),
    # ConstantKernel() * EpanechnikovKernel() + WhiteKernel(),
    Matern(nu = 0.1) + EpanechnikovKernel(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    Matern(nu = 1.1) + EpanechnikovKernel(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    Matern(nu = 2.0) + EpanechnikovKernel(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
DotProduct() + Matern(nu=0.1) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    DotProduct() + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    # Matern(nu = 2.5) + EpanechnikovKernel(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    # Matern(nu = 1.5) + EpanechnikovKernel(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    # DotProduct() + RBF(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    # DotProduct() + EpanechnikovKernel(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    # DotProduct() + Matern(nu=1.5) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    # DotProduct() + Matern(nu=2.5) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel
    # hybridGPKernal(),
    # EpanechnikovKernel(length_scale=1.0) + WhiteKernel(noise_level=0.1),
    # Matern(nu = 2.5) + EpanechnikovKernel(length_scale=1.0) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5, 1e5)) + seasonal_kernel + irregularities_kernel,
    # Matern(nu = 1.5) + EpanechnikovKernel(length_scale=1.0) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5, 1e5)) + seasonal_kernel + irregularities_kernel
]

for filename in os.listdir(folder_path):
    print(f"Processing {filename}")
    if not filename.endswith('.csv'):
        continue

    # Load data
    df = pd.read_csv(os.path.join(folder_path, filename))
    v = filename.split('.csv')[0]

    # Split data
    df1 = df.loc[~df['location'].isin(test_location)]
    df2 = df.loc[df['location'].isin(test_location)].sort_values(by=['location'])
    X_train, y_train, X_test, y_test = testAndTrainData(df1, df2)

    # Feature engineering: Add distance-based features
    X_train_dist = calculate_distances(X_train)
    X_test_dist = calculate_distances(X_test)

    # Combine coordinates with distance features
    X_train_extended = np.hstack((X_train, X_train_dist))
    X_test_extended = np.hstack((X_test, X_test_dist))

    # Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_extended)
    X_test_scaled = scaler.transform(X_test_extended)

    # Track best model for this beacon
    best_mae = float('inf')
    best_gp = None
    best_kernel = None

    # Try each kernel candidate
    for kernel in kernel_candidates:
        gp = GaussianProcessRegressor(
            kernel=kernel,
            n_restarts_optimizer=15,
            normalize_y=True,
            alpha=0.01  # Small noise for stability
        )

        # Fit and predict
        gp.fit(X_train_scaled, y_train)
        y_pred, _ = gp.predict(X_test_scaled, return_std=True)

        # Calculate MAE
        current_mae = mean_absolute_error(y_test, y_pred)

        # Track best kernel
        if current_mae < best_mae:
            best_mae = current_mae
            best_gp = gp
            best_kernel = kernel

    print(f"Best kernel for {v}: {best_kernel} with MAE: {best_mae:.2f}")

    # ================== DYNAMIC MESHGRID GENERATION ==================
    # Combine all data points to determine bounds
    all_points = np.vstack((X_train, X_test))

    # Get min/max coordinates with buffer
    min_x, min_y = np.min(all_points, axis=0)
    max_x, max_y = np.max(all_points, axis=0)

    # Add 10% buffer to the bounding box
    x_buffer = max(1, int(0.1 * (max_x - min_x)))
    y_buffer = max(1, int(0.1 * (max_y - min_y)))

    # Define bounds with buffer (clamped to 1-242/259 range)
    min_x = max(1, min_x - x_buffer)
    max_x = min(194, max_x + x_buffer)
    min_y = max(1, min_y - y_buffer)
    max_y = min(230, max_y + y_buffer)

    # Create dynamic meshgrid within bounds
    x1 = np.arange(min_x, max_x + 1).astype(int)
    x2 = np.arange(min_y, max_y + 1).astype(int)
    X0p, X1p = np.meshgrid(x1, x2)
    grid_points = np.c_[X0p.ravel(), X1p.ravel()]

    # Add distance features to grid points
    grid_distances = calculate_distances(grid_points)
    grid_points_extended = np.hstack((grid_points, grid_distances))
    grid_points_scaled = scaler.transform(grid_points_extended)

    # Predict using best model
    mean_pred, std_pred = best_gp.predict(grid_points_scaled, return_std=True)

    # Create full grid arrays with -99 default
    full_x = np.arange(1, 195)
    full_y = np.arange(1, 231)
    full_X0p, full_X1p = np.meshgrid(full_x, full_y)
    full_grid_points = np.c_[full_X0p.ravel(), full_X1p.ravel()]

    # Initialize full RSSI arrays with -99
    full_mean_pred = np.full(len(full_grid_points), -104.0)
    full_std_pred = np.full(len(full_grid_points), -104.0)

    # Create mapping from coordinate to index
    coord_to_index = {(x, y): i for i, (x, y) in enumerate(full_grid_points)}

    # Map predictions to full grid
    for (x, y), mean_val, std_val in zip(grid_points, mean_pred, std_pred):
        idx = coord_to_index.get((x, y), None)
        if idx is not None:
            full_mean_pred[idx] = mean_val
            full_std_pred[idx] = std_val

    # Prepare data for saving
    data = [
        [f"{x}, {y}", round(mean, 2), round(std, 2)]
        for (x, y), mean, std in zip(full_grid_points, full_mean_pred, full_std_pred)
    ]
    # ================== END DYNAMIC MESHGRID ==================

    df_out = pd.DataFrame(data, columns=["location", "rssi_mean", "rssi_std"])
    df_out.to_csv(f'/content/Localization_project-main/Third/beacons_pd/{v}.csv', index=False)

    # Final evaluation
    y_pred, _ = best_gp.predict(X_test_scaled, return_std=True)
    final_mae = mean_absolute_error(y_test, y_pred)
    print(f"  Final MAE: {final_mae:.2f}")
    print(f"  Grid size: Original (194x230) | Dynamic: ({min_x}-{max_x})x({min_y}-{max_y})")

Processing IW25030937.csv


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k1__k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__k1__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gp

Best kernel for IW25030937: Matern(length_scale=1, nu=1.1) + EpanechnikovKernel(length_scale=1.0) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 0.80


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:478: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn(


  Final MAE: 0.80
  Grid size: Original (194x230) | Dynamic: (67-122)x(22-58)
Processing IW25030972.csv


...

Best kernel for IW25030193: Matern(length_scale=1, nu=1.1) + EpanechnikovKernel(length_scale=1.0) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 4.36
  Final MAE: 4.36
  Grid size: Original (194x230) | Dynamic: (61-73)x(145-195)
Processing IW25030959.csv


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.11/dist-pa

Best kernel for IW25030959: DotProduct(sigma_0=1) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 2.86
  Final MAE: 2.86
  Grid size: Original (194x230) | Dynamic: (168-194)x(188-230)
Processing IW25030949.csv


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k2__k1__constant_value is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__k1__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/

Best kernel for IW25030949: Matern(length_scale=1, nu=0.1) + EpanechnikovKernel(length_scale=1.0) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 7.47
  Final MAE: 7.47
  Grid size: Original (194x230) | Dynamic: (68-75)x(44-184)


Localization_2

In [ ]:
#!/usr/bin/env python
import os
from IPython.display import display
from itertools import product
import numpy as np
from sklearn.metrics import mean_absolute_error
import pandas as pd
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import accuracy_score
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C,WhiteKernel , RationalQuadratic ,ExpSineSquared
# %matplotlib notebook


# Directory containing beacon files
def GroundTruthData(directory_gt, floor,test_location,num_files):

    # Initialize an empty list to hold dataframes
    dataframes_gt = []
    # Loop through each beacon file
    for filename in os.listdir(directory_gt):
        # print(filename)
        file_path = os.path.join(directory_gt, filename)
        df = pd.read_csv(file_path)
        df.rename(columns={'rssi_mean':  filename.split('.csv')[0]}, inplace=True)
        #df = df.drop(['rssi_std'], axis=1) # Drop 'rssi_std' column
        dataframes_gt.append(df)  # Append dataframe to the list

    dataframe_gt = pd.concat([df.set_index('location') for df in dataframes_gt], axis=1, join='outer').reset_index()
    dataframe_gt.fillna('-100.7', inplace=True)
    # print(dataframe_gt.round(0))
    dataframe_gt.to_csv(floor+'/all_scan_sample.csv', index=False)

    ### Randomly sampled 6 test location out of total 29 location and 3 outlier = 32 points####
    test_data = dataframe_gt.loc[dataframe_gt['location'].isin(test_location)]
    test_data = test_data.sort_values(by=['location'])
    temp = test_data.location.str.split(",", n = -1, expand = True)
    # print(dataframe_gt)
    test_data['cordX'] = temp[0].astype(int)
    test_data['cordY'] = temp[1].astype(int)
    # print(data.head())
    X_gt= test_data.iloc[:, 1:num_files]
    y1_gt = test_data['cordX']
    y2_gt = test_data['cordY']
    test_data.head()
    test_data.shape

    return X_gt, y1_gt, y2_gt


def Custom_PD_data(directory_pd, floor,test_location,num_files):

    # Initialize an empty list to hold dataframes
    dataframes_pd = []
    # Loop through each beacon file
    for filename in os.listdir(directory_pd):
        # print(filename)
        file_path = os.path.join(directory_pd, filename)
        df = pd.read_csv(file_path)
        df.rename(columns={'rssi_mean':  filename.split('.csv')[0]}, inplace=True)
        df = df.drop(['rssi_std'], axis=1) # Drop 'rssi_std' column
        dataframes_pd.append(df)  # Append dataframe to the list

    dataframe_pd = pd.concat([df.set_index('location') for df in dataframes_pd], axis=1, join='inner').reset_index()

    ### Randomly sampled 6 test location out of total 29 location and 3 outlier = 32 points####
    test_data = dataframe_pd.loc[dataframe_pd['location'].isin(test_location)]
    test_data = test_data.sort_values(by=['location'])
    temp = test_data.location.str.split(",", n = -1, expand = True)
    test_data['cordX'] = temp[0].astype(int)
    test_data['cordY'] = temp[1].astype(int)
    # print(data.head())
    X_pd_custom= test_data.iloc[:, 1:num_files]
    y1_pd_custom = test_data['cordX']
    y2_pd_custom = test_data['cordY']
    test_data.head()
    test_data.shape

    return X_pd_custom, y1_pd_custom, y2_pd_custom

### beacons_pd folder include predicted beacons rssi data ########
def GPPredictedData(directory_pd, num_files):
    dataframes_pd = []
    for filename in os.listdir(directory_pd):
        file_path = os.path.join(directory_pd, filename)
        # Read CSV file
        df = pd.read_csv(file_path)
        # Rename column
        df.rename(columns={'rssi_mean':  filename.split('.csv')[0]}, inplace=True)
        # Drop 'rssi_std' column
        df = df.drop(['rssi_std'], axis=1)
        # Append dataframe to the list
        dataframes_pd.append(df)
    # Merge all dataframes on 'location'
    dataframes_pd = pd.concat([df.set_index('location') for df in dataframes_pd], axis=1, join='inner').reset_index()
    new = dataframes_pd.location.str.split(",", n = -1, expand = True)
    dataframes_pd['cordX'] = new[0].astype(int)
    dataframes_pd['cordY'] = new[1].astype(int)
    X_pd = dataframes_pd.iloc[:, 1:num_files]
    y1_pd = dataframes_pd['cordX']
    y2_pd = dataframes_pd['cordY']

    dataframes_pd.head()
    dataframes_pd.shape

    return X_pd, y1_pd, y2_pd

#################### KNN training for X-axis ####################
def knnXaxis(X_train, y1_train, X_test,y1_test, y1_gt):
    print("---------KNN training for X-axis result---------" )
    # print(X_test, y1_test)
    knn_classifier = KNeighborsClassifier(n_neighbors = 3)
    # print(X_train, y1_train)
    knn_classifier.fit(X_train, y1_train.values.ravel())
    # test using GP predicted test dataset
    y1_pred=knn_classifier.predict(X_test)
    y1_pred_custom=knn_classifier.predict(X_pd_custom)
    # print(y1_pred_custom)
    # print(y1_pd_custom)
    # knn_accuracy=accuracy_score(y1_test, y1_pred)
    # print('\nThe Models Accuracy is', knn_accuracy)
    MAE1 = mean_absolute_error(y1_pred,y1_test)
    MAE2 = mean_absolute_error(y1_pred_custom,y1_pd_custom)
    print("Predicted data knn-mae-X-axis :\t",MAE1*0.305, "m")
    print("Custom Predicted data knn-mae-X-axis (0 to 15 ft):\t",MAE2*0.305, "m")
    # test using ground truth test dataset
    # print("Inputs==========",X_gt)
    y1_pred_gt=knn_classifier.predict(X_gt)
    # print("Outputs======",y1_pred_gt)
    # # knn_accuracy=accuracy_score(y1_gt, y1_pred_gt)
    # # print('\nThe Models Accuracy for ground truth values, is', knn_accuracy)
    # print(y1_gt,y1_pred_gt)
    MAE3 = mean_absolute_error(y1_pred_gt,y1_gt)
    print("Ground truth data knn-mae-X-axis :\t",MAE3*0.305, "m")

    return MAE1, MAE2, MAE3, y1_pred, y1_pred_custom, y1_pred_gt


#################### KNN training for Y-axis ####################
def knnYaxis(X_train, y2_train, X_test, y2_test,y2_gt):
    print("---------KNN training for Y-axis result---------" )
    knn_classifier = KNeighborsClassifier(n_neighbors = 3)
    knn_classifier.fit(X_train, y2_train.values.ravel())
    # test using GP predicted test dataset
    y2_pred=knn_classifier.predict(X_test)
    y2_pred_custom=knn_classifier.predict(X_pd_custom)
    #print(y2_pred)
    # knn_accuracy=accuracy_score(y2_test, y2_pred)
    # print('\nThe Models Accuracy is', knn_accuracy)

    MAE4 = mean_absolute_error(y2_pred,y2_test)
    MAE5 = mean_absolute_error(y2_pred_custom,y2_pd_custom)
    print("Predicted data knn-mae-Y-axis :\t",MAE4*0.305, "m")
    print("Custom Predicted data knn-mae-Y-axis (0 to 15 ft):\t",MAE5*0.305, "m")
    # test using ground truth test dataset
    y2_pred_gt=knn_classifier.predict(X_gt)
    MAE6 = mean_absolute_error(y2_pred_gt,y2_gt)
    # print(y2_gt,y2_pred_gt)
    print("Ground truth data knn-mae-Y-axis :\t",MAE6*0.305, "m")

    return MAE4, MAE5, MAE6, y2_pred, y2_pred_custom, y2_pred_gt


########## KNN - Localization performance #########################
########### Part 1: Result on GP predicted test dataset ##################
def knnLocalizationPredictedData(y1_test, y2_test, X_test, y1_pred, y2_pred, path_3):
    r1 = pd.concat([y1_test, y2_test, X_test], axis=1)
    # r1.rename(columns={'cordX': 'cordXold', 1: 'cordYold'}, inplace=True)
    # r1.drop(columns=["cordX", "cordY"], inplace=True)
    r2 = pd.concat([pd.DataFrame({'SP_cordX':y1_pred}),pd.DataFrame({'SP_cordY':y2_pred})], axis=1)
    result = pd.concat([r1,r2.set_index(r1.index)], axis=1)
    #display(result)
    # euclidean distance error in a grid world
    # result = result.rename(columns={0: 'cordXold', 1: 'cordYold'}, inplace=True)
    result['dist_localization_error_pd']  = np.round(np.sqrt((result['cordX'] - result['SP_cordX'])**2 + (result['cordY'] - result['SP_cordY'])**2),2)
    result['loc'] = (result['cordX'].map(str)).str.cat(result['cordY'].map(str),sep=",")
    result['Pred_loc'] = (result['SP_cordX'].map(str)).str.cat(result['SP_cordY'].map(str),sep=",")
    result = result[['loc','Pred_loc','dist_localization_error_pd']]
    # print("Part 1: Result on GP predicted test dataset ==========")

    print("Localziation_mse_knn_pd_predicted", np.mean(result['dist_localization_error_pd'])*0.305, "m")
    # print("Localization error WRT Predicted data", result['dist_localization_error_pd']*0.305, "m")
    #result.to_csv(path_3+'/knn_result_pd_predicted.csv', index=False)

    return np.mean(result['dist_localization_error_pd'])


def knnLocalizationcustomPD(y1_pd_custom, y2_pd_custom, X_pd_custom, y1_pred_custom, y2_pred_custom):
    r1 = pd.concat([y1_pd_custom, y2_pd_custom, X_pd_custom], axis=1)
    # r1.rename(columns={'cordX': 'cordXold', 1: 'cordYold'}, inplace=True)
    # r1.drop(columns=["cordX", "cordY"], inplace=True)
    r2 = pd.concat([pd.DataFrame({'SP_cordX':y1_pred_custom}),pd.DataFrame({'SP_cordY':y2_pred_custom})], axis=1)
    result = pd.concat([r1,r2.set_index(r1.index)], axis=1)
    #display(result)
    # euclidean distance error in a grid world
    # result = result.rename(columns={0: 'cordXold', 1: 'cordYold'}, inplace=True)
    result['dist_localization_error_pd']  = np.round(np.sqrt((result['cordX'] - result['SP_cordX'])**2 + (result['cordY'] - result['SP_cordY'])**2),2)
    result['loc'] = (result['cordX'].map(str)).str.cat(result['cordY'].map(str),sep=",")
    result['Pred_loc'] = (result['SP_cordX'].map(str)).str.cat(result['SP_cordY'].map(str),sep=",")
    result = result[['loc','Pred_loc','dist_localization_error_pd']]
    # print("Part 1: Result on GP predicted test dataset ==========")

    print("Localziation_mse_knn_custom_pd_predicted (0 to 15 ft)", np.mean(result['dist_localization_error_pd'])*0.305, "m")
    # print("Localization error WRT Predicted data", result['dist_localization_error_pd']*0.305, "m")
    #result.to_csv(path_3+'/knn_result_pd_predicted.csv', index=False)

    return np.mean(result['dist_localization_error_pd'])



########## Part 2: Result on Ground truth test dataset ###################
def knnLocalizationGroundTruthData(y1_gt, y2_gt, X_gt, y1_pred_gt, y2_pred_gt, path_4):
    r1 = pd.concat([y1_gt, y2_gt, X_gt], axis=1)
    r2 = pd.concat([pd.DataFrame({'GPP_cordX':y1_pred_gt}),pd.DataFrame({'GPP_cordY':y2_pred_gt})], axis=1)
    result = pd.concat([r1,r2.set_index(r1.index)], axis=1)
    #print(result)
    # euclidean distance error in a grid world
    result['dist_localization_error_gt']  = np.round(np.sqrt((result['cordX'] - result['GPP_cordX'])**2 + (result['cordY'] - result['GPP_cordY'])**2),2)
    result['loc'] = (result['cordX'].map(str)).str.cat(result['cordY'].map(str),sep=",")
    result['Pred_loc'] = (result['GPP_cordX'].map(str)).str.cat(result['GPP_cordY'].map(str),sep=",")
    result = result[['loc','Pred_loc','dist_localization_error_gt']]
    # print("Part 2: Result on Ground truth test dataset ==========")

    print("Localziation_mse_knn_gt_predicted", np.mean(result['dist_localization_error_gt'])*0.305, "m")
    # print("Localization error WRT Ground truth data", result['dist_localization_error_gt']*0.305, "m")
    #result.to_csv(path_4+'/knn_result_gt_predicted.csv', index=False)

    return np.mean(result['dist_localization_error_gt'])


floor = '/content/Localization_project-main/Third'
path_1 ='/content/Localization_project-main/Third/beacons_gt'

test_location_pd = [
    "174, 167", "105, 191", "118, 194", "70, 121", "71, 27", "75, 71", "51, 40", "87, 54", "30, 27", "157, 56", "56, 34", "154, 189", "55, 43", "31, 60", "81, 40", "71, 141", "32, 33", "79, 187", "174, 136", "196, 226", "111, 32", "18, 54", "79, 21", "78, 28", "50, 32", "167, 230", "66, 32", "107, 58", "152, 56", "146, 193", "80, 31", "58, 47", "64, 82", "170, 214", "95, 22", "68, 135", "73, 160", "140, 195", "129, 56", "149, 53", "73, 170", "96, 190", "68, 94", "40, 28", "29, 36", "91, 191", "20, 54", "69, 129", "172, 209", "75, 32", "102, 28", "169, 214", "71, 63", "41, 200", "75, 76", "29, 26", "69, 130", "71, 64", "43, 192", "69, 41", "136, 62", "113, 32", "103, 57", "104, 26", "181, 232", "108, 21", "73, 113", "79, 37", "77, 149", "65, 56", "97, 189", "197, 221", "72, 150", "71, 120", "90, 28", "134, 55", "141, 193", "73, 115", "65, 194", "73, 176", "48, 36", "64, 194", "72, 114", "149, 187", "167, 185", "110, 20", "101, 26", "180, 233", "175, 220", "61, 194", "75, 161", "168, 182", "76, 107", "114, 54", "74, 69", "199, 222", "65, 55", "80, 194", "33, 28", "57, 47",
    "80, 144", "14, 50", "179, 136", "80, 88", "50, 58", "80, 132", "77, 60", "52, 61", "14, 29", "127, 43", "138, 56", "117, 63", "102, 198", "147, 59", "123, 28", "55, 185", "77, 115", "178, 212", "32, 48", "178, 134", "62, 24", "145, 49", "3, 49", "96, 17", "99, 198", "78, 74", "179, 177", "147, 61", "37, 46", "15, 48", "135, 200", "38, 50", "112, 38", "43, 184", "83, 197", "50, 53", "22, 21", "70, 47", "110, 40", "3, 62", "61, 62", "13, 26", "99, 47", "39, 49", "80, 92", "48, 44", "20, 22", "77, 183", "169, 232", "108, 46", "49, 183", "180, 204", "63, 42", "80, 157", "181, 221", "78, 46", "64, 42", "88, 62", "174, 236", "65, 90", "58, 183", "79, 75", "81, 102", "77, 114", "75, 16", "124, 51", "61, 87", "36, 63", "77, 162", "106, 184", "78, 180", "53, 185", "61, 63", "66, 179", "84, 17", "103, 16", "133, 199", "36, 189", "61, 49", "125, 44", "118, 200", "108, 185", "165, 171", "130, 196", "22, 23", "174, 196", "63, 131", "69, 138", "136, 51", "75, 119", "86, 199", "175, 231", "170, 233", "51, 62", "19, 20", "87, 29", "63, 161", "18, 45", "78, 17", "179, 238",
    "116, 13", "51, 18", "15, 16", "62, 180", "146, 63", "84, 95", "165, 53", "161, 180", "33, 202", "161, 202", "183, 138", "32, 190", "84, 84", "31, 15", "62, 71", "52, 17", "143, 63", "58, 82", "32, 191", "126, 68", "8, 29", "116, 201", "82, 137", "123, 14", "139, 180", "82, 90", "84, 149", "103, 12", "61, 99", "124, 180", "37, 208", "53, 203", "135, 203", "60, 67", "130, 202", "90, 37", "61, 172", "144, 50", "104, 202", "183, 146", "83, 140", "183, 206", "60, 171", "116, 15", "9, 67", "38, 18", "75, 13", "41, 43", "181, 207", "129, 26", "159, 179", "46, 180", "82, 181", "59, 16", "38, 66", "167, 130", "121, 180", "128, 26", "132, 203", "130, 20", "162, 235", "60, 103", "52, 16", "161, 132", "83, 79", "141, 68", "34, 188", "161, 224", "163, 59", "8, 25", "60, 154", "129, 45", "116, 14", "161, 157", "84, 122", "44, 42", "124, 179", "160, 179", "129, 48", "105, 65", "183, 147", "83, 152", "178, 131", "125, 180", "121, 14", "163, 58", "59, 81", "74, 13", "85, 179", "84, 87", "152, 202", "103, 203", "61, 116", "120, 179", "154, 67", "128, 23", "46, 209", "159, 163", "134, 48", "62, 159"
]
test_location_ground = ["79, 24", "79, 32", "71, 38", "71, 28", "71, 36", "71, 32", "55, 38", "55, 46", "71, 30", "77, 38", "94, 55", "72, 55", "65, 55", "16, 55", "31, 32", "62, 32", "55, 32", "47, 32", "40, 32", "19, 32", "24, 38", "55, 42", "8, 55", "77, 32", "19, 28", "31, 28", "55, 28", "74, 55", "84, 55", "93, 25", "24, 32", "21, 55", "35, 55", "55, 55", "45, 55", "31, 55", "72, 65", "118, 55", "104, 55", "114, 55", "118, 37", "72, 59", "72, 70", "69, 82", "72, 88", "115, 25", "118, 46", "118, 25", "133, 58", "111, 25", "118, 41", "116, 32", "72, 75", "72, 93", "72, 106", "72, 113", "72, 124", "72, 143", "72, 155", "72, 82", "72, 99", "69, 124", "72, 149", "153, 55", "72, 132", "72, 163", "72, 173", "72, 174", "72, 186", "62, 191", "72, 191", "76, 191", "171, 210", "81, 191", "89, 191", "115, 191", "99, 191", "108, 191", "44, 191", "125, 191", "142, 191", "155, 191", "163, 191", "171, 191", "52, 191", "85, 191", "93, 191", "103, 191", "119, 191", "135, 191", "203, 229", "203, 219", "203, 214", "148, 191", "171, 201", "159, 191", "44, 198", "171, 181", "171, 172", "171, 161", "171, 151", "171, 185", "171, 176", "171, 166", "171, 220", "171, 227", "181, 230", "189, 230", "194, 230", "99, 25", "106, 23", "171, 140", "171, 156", "171, 146", "194, 222"]

num_files = len(os.listdir(path_1))+1    # Count the number of files
test_location = test_location_ground


#print(test_location)
X_gt, y1_gt, y2_gt = GroundTruthData(path_1, floor, test_location,num_files)

path_2 = '/content/Localization_project-main/Third/beacons_pd'
num_files = len(os.listdir(path_2))+1    # Count the number of files
X_pd_custom, y1_pd_custom, y2_pd_custom = Custom_PD_data(path_2,floor,test_location_pd,num_files)
X_pd, y1_pd, y2_pd = GPPredictedData(path_2,num_files)
#print(num_files)
#print(X_pd_custom)
# #### Split the data set for training and testing ###########
X_train, X_test, y1_train, y1_test = train_test_split(X_pd, y1_pd,test_size=0.2, random_state= 7)
mae_pd_x, mae_pd_custom_x, mae_gt_x, y1_pred, y1_pred_custom, y1_pred_gt = knnXaxis(X_train, y1_train, X_test,y1_test, y1_gt)
#print(y1_test)
# print(X_pd)
# #### Split the data set for training and testing ###########
X_train, X_test, y2_train, y2_test = train_test_split(X_pd, y2_pd,test_size=0.2, random_state= 7)
mae_pd_y, mae_pd_custom_y, mae_gt_y, y2_pred, y2_pred_custom, y2_pred_gt =  knnYaxis(X_train, y2_train, X_test, y2_test,y2_gt)
#print(y1_test,y2_test)
# print(y1_gt.to_frame()['cordX'], y2_gt.to_frame()['cordY'])
#y1_y2_gt = pd.concat([y1_gt.to_frame(), y2_gt.to_frame()], axis=1)
# print(y1_y2_gt)
y1_y2_pd_gt = pd.concat([pd.DataFrame(y1_pred_gt), pd.DataFrame(y2_pred_gt)], axis=1)
y1_y2_pd_gt.rename(columns={'0': 'cordX', '0': 'cordY'})
# print(pd.DataFrame(y1_pred_gt))
# print(y1_y2_pd_gt)
# print(pd.concat([y1_y2_gt, y1_y2_pd_gt], axis=1))
print("---------KNN Localization performance evaluatiion ---------" )
path_3 = floor+'/results'
path_4 = floor+'/results'
mse_knn_pd = knnLocalizationPredictedData(y1_test, y2_test, X_test, y1_pred, y2_pred, path_3)
mse_knn_custom_pd = knnLocalizationcustomPD(y1_pd_custom, y2_pd_custom, X_pd_custom, y1_pred_custom, y2_pred_custom)
mse_knn_gt = knnLocalizationGroundTruthData(y1_gt, y2_gt, X_gt, y1_pred_gt, y2_pred_gt, path_4)

/tmp/ipython-input-3-1067822134.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-100.7' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataframe_gt.fillna('-100.7', inplace=True)


---------KNN training for X-axis result---------
Predicted data knn-mae-X-axis :	 11.291117772299415 m
Custom Predicted data knn-mae-X-axis (0 to 15 ft):	 4.130692041522491 m
Ground truth data knn-mae-X-axis :	 1.6072173913043477 m
---------KNN training for Y-axis result---------
Predicted data knn-mae-Y-axis :	 9.799680076199012 m
Custom Predicted data knn-mae-Y-axis (0 to 15 ft):	 2.957128027681661 m
Ground truth data knn-mae-Y-axis :	 1.3022173913043478 m
---------KNN Localization performance evaluatiion ---------
Localziation_mse_knn_pd_predicted 16.64125934558494 m
Localziation_mse_knn_custom_pd_predicted (0 to 15 ft) 5.7199425605536325 m
Localziation_mse_knn_gt_predicted 2.5383426086956526 m
